<a href="https://colab.research.google.com/github/rafabrnd/Coursera_Capstone/blob/master/coursera_capstone_Kmeans_torrontoneighborhoods.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Segmenting and Clustering Neighborhoods in Toronto**
### *Applied Data Science Capstone > Week 3*






<img src="https://media.giphy.com/media/olAik8MhYOB9K/giphy.gif" width="250" align="left">



In [0]:
import numpy as np
import pandas as pd
print("Hello Capstone Project Course")

Hello Capstone Project Course
